In [72]:
import os
import re
import numpy
import datetime
import nltk
from subprocess import Popen,PIPE
# To print all outputs, not just the last one
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## Most frequent tagging , NLTK tagging

In [82]:
class BrownCorpus(object):
    def __init__(self,dirname):
        self.dirname = dirname
        self.pos_ct = {}
        self.final_pos = {}
        self.final_ct = {}
        total = len(os.listdir(self.dirname))
        self.train_size = int(total*0.8)
        self.test_size = total-self.train_size
    def compute_pos_ct(self):
        for indx,fname in enumerate(os.listdir(self.dirname)):
            # Run on training set
            if indx>=self.train_size:
                break
            if re.match(r'c.+',fname)==None:
                continue
            fname = os.path.join(self.dirname,fname)
            if not os.path.isfile(fname):
                continue
            for line in open(fname):
                token_tags = [t.split('/') for t in line.split() if len(t.split('/'))==2]
                for token,tag in token_tags:
                    token = str(token).lower()
                    try:
                        self.pos_ct[(token,tag)] +=1
                    except:
                        self.pos_ct[(token,tag)] = 1
    def build_pos_oracle(self):
        for token_tag in self.pos_ct:
            ct = self.pos_ct[token_tag]
            token = token_tag[0]
            tag = token_tag[1]
            try:
                if self.final_ct[token]<ct:
                    self.final_pos[token]=tag
                    self.final_ct[token]=ct
            except:
                self.final_pos[token]=tag
                self.final_ct[token]=ct
    def get_accuracy(self):
        total_tokens = 0
        error_ct = 0
        for indx,fname in enumerate(os.listdir(self.dirname)):
            if indx<self.train_size:
                continue
            if re.match(r'c.+',fname)==None:
                continue
            fname = os.path.join(self.dirname,fname)
            if not os.path.isfile(fname):
                continue
            for line in open(fname):
                token_tags = [t.split('/') for t in line.split() if len(t.split('/'))==2]
                total_tokens += len(token_tags)
                for token,tag in token_tags:
                    token = str(token).lower()
                    try:
                        if tag!=self.final_pos[token]:
                            error_ct+=1
                    except:
                        pass
        return 1-error_ct/float(total_tokens)
    def nltk_tagger_accuracy(self):
        total_tokens = 0
        error_ct = 0
        for indx,fname in enumerate(os.listdir(self.dirname)):
            if indx<self.train_size:
                continue
            if re.match(r'c.+',fname)==None:
                continue
            fname = os.path.join(self.dirname,fname)
            if not os.path.isfile(fname):
                continue
            for line in open(fname):
                token_tags = [nltk.tag.str2tuple(t) for t in line.split() if len(t.split('/'))==2]
                total_tokens += len(token_tags)
                tokens = [tt[0] for tt in token_tags]
                tokens_txt = " ".join(tokens)
                nltk_tags = nltk.pos_tag(tokens_txt)
                for i in range(len(token_tags)):
                    nltk_tag = nltk_tags[i][1]
                    given_tag = token_tags[i][1]
                    if nltk_tag.lower()[:2] != given_tag.lower()[:2]:
                        error_ct += 1
        return 1-error_ct/float(total_tokens)
    def opennlp_tagger_accuracy(self):
        total_tokens = 0
        error_ct = 0
        outfile = open('opennlp_tag','w')
        doc = ""
        for indx,fname in enumerate(os.listdir(self.dirname)):
            if indx<self.train_size:
                continue
            if re.match(r'c.+',fname)==None:
                continue
            fname = os.path.join(self.dirname,fname)
            if not os.path.isfile(fname):
                continue
            for line in open(fname):
                token_tags = [nltk.tag.str2tuple(t) for t in line.split() if len(t.split('/'))==2]
                total_tokens += len(token_tags)
                tokens = [tt[0] for tt in token_tags]
                tokens_txt = " ".join(tokens)
                doc += tokens_txt
                doc += "\n"
            
                

In [83]:
brown = BrownCorpus('/home/debojyoti/brown/')
brown.compute_pos_ct()
brown.build_pos_oracle()
brown.get_accuracy()
brown.nltk_tagger_accuracy()

2016-10-16 21:05:47.238152
2016-10-16 21:05:48.028085
2016-10-16 21:05:48.075190


0.8422427445175061

In [118]:
a=[1,2,3]
a.extend(a)
print a

[1, 2, 3, 1, 2, 3]


## OpenNLP tagger

In [128]:
from subprocess import Popen,PIPE,STDOUT
# os.system('../apache-opennlp-1.6.0/bin/opennlp POSTagger en-pos-perceptron.bin < /home/debojyoti/brown/ca01')
doc = ""
tags_all = []
total_tokens = 0
err_ct = 0
with open('/home/debojyoti/brown/ca01') as infile:
    for line in infile:
        token_tags = [nltk.tag.str2tuple(t) for t in line.split() if len(t.split('/'))==2]
        tokens = [tt[0] for tt in token_tags]
        tags = [tt[1] for tt in token_tags]
        tags_all.extend(tags)
        tokens_txt = " ".join(tokens)
        doc += tokens_txt
        doc += "\n"    
    cmd = ['../apache-opennlp-1.6.0/bin/opennlp', 'POSTagger', '../apache-opennlp-1.6.0/bin/en-pos-perceptron.bin']
    myprogram = Popen(cmd,stdin=PIPE,stdout=PIPE,stderr=STDOUT)
    stdout = myprogram.communicate(input=doc)[0]
    stdout = stdout.split('\n',1)[1]
    print stdout[:20]
    for line in stdout:
        try:
            print line
            tags = [t.split('_')[1] for t in line.split()]
        except:
            print line
            raise
        total_tokens += len(tags)
        for indx,tag in enumerate(tags):
            if tag.lower()!=tags_all[i]:
                err_ct += 1
print total_tokens,err_ct



The_DT Fulton_NNP 




T
T


IndexError: list index out of range

In [96]:
x="abc"
x+x

'abcabc'